# Generate word2vec model - include all new topic words

In [1]:
from gensim.models import Word2Vec, KeyedVectors
import gensim
import time
import numpy as np
from tqdm import tqdm
from pprint import pprint
import pickle
import spacy
import re
import operator

In [2]:
nlp = spacy.load('pl_spacy_model')

In [32]:
# set variables 
LANG = 'pl'
MAX_WORDS = 500000  # maximum processed words to small model 

MODEL_DIR = '/mnt/data/data/models-fasttext/'
TOPICS_DIR = '/home/dzon/kajo/topics/topic_lists-based_on_en-US/'

model_filename = MODEL_DIR + 'cc.pl.300-lemma_500k_min2_v2.vec.bin'

excluded_model_name = MODEL_DIR + 'cc.pl.300-lemma_500k_min2_excluded.vec.bin'

model_orig_filename = MODEL_DIR + 'cc.pl.300.vec.bin'
model_orig_filename_vec = MODEL_DIR + 'cc.pl.300.vec.gz'

pickled_words_lemmas = MODEL_DIR + LANG + '_ft_all_words_lemmas.pickle'

lemma_list_file = TOPICS_DIR + 'topics_lemma_'+ LANG +'.lst'
nolemma_list_file = TOPICS_DIR + 'topics_'+ LANG +'.lst'

excluded_filename = TOPICS_DIR + 'topics_' + LANG + '-not_in_model.lst'

lemma_list_file_excluded = TOPICS_DIR + 'topics_lemma_excluded_' + LANG + '.lst'
nolemma_list_file_excluded = TOPICS_DIR + 'topics_excluded_' + LANG + '.lst'


In [4]:
# load big model
model_big = KeyedVectors.load_word2vec_format(model_orig_filename, binary=True)

In [5]:
#load pickled lemma list
with open(pickled_words_lemmas, 'rb') as f:
    all_words_lemmas = pickle.load(f)
pprint(all_words_lemmas[:50])

[('w', 'w'),
 ('i', 'i'),
 ('z', 'z'),
 ('na', 'na'),
 ('do', 'do'),
 ('się', 'się'),
 ('nie', 'nie'),
 ('to', 'to'),
 ('jest', 'być'),
 ('o', 'o'),
 ('W', 'w'),
 ('a', 'a'),
 ('że', 'że'),
 ('od', 'od'),
 ('dla', 'dla'),
 ('przez', 'przez'),
 ('roku', 'rok'),
 ('oraz', 'oraz'),
 ('za', 'za'),
 ('jak', 'jak'),
 ('po', 'po'),
 ('tym', 'to'),
 ('są', 'być'),
 ('lub', 'lub'),
 ('co', 'co'),
 ('ale', 'ale'),
 ('ze', 'z'),
 ('jako', 'jako'),
 ('tego', 'to'),
 ('ma', 'mieć'),
 ('_', '_'),
 ('czy', 'czy'),
 ('które', 'który'),
 ('I', 'i'),
 ('tylko', 'tylko'),
 ('tak', 'tak'),
 ('jego', 'on'),
 ('szczegółowo', 'szczegółowo'),
 ('już', 'już'),
 ('Bardziej', 'bardzo'),
 ('pod', 'pod'),
 ('może', 'może'),
 ('przy', 'przy'),
 ('Na', 'na'),
 ('także', 'także'),
 ('który', 'który'),
 ('bardzo', 'bardzo'),
 ('był', 'być'),
 ('został', 'zostać'),
 ('zł', 'złoty')]


In [6]:
# generate helpful dictionaries/lists
big_lemmas_set = set([l for (_,l) in all_words_lemmas])
print(len(big_lemmas_set))

big_words_set = set([w for (w,_) in all_words_lemmas])
print(len(big_words_set))

lemmas = {}
words_lower_orig = {}
big_lemma_word_dic = {}
big_word_lemma_dic = {}
for (w,l) in all_words_lemmas:
    big_word_lemma_dic[w] = l
    
    #dictionary of lowercase word -> all original words
    if w.lower() in words_lower_orig:
        words_lower_orig[w.lower()].append(w)
    else:
        words_lower_orig[w.lower()] = [w]
        
    if l in big_lemma_word_dic:
        big_lemma_word_dic[l].append(w)
        lemmas[l] += 1
    else:
        big_lemma_word_dic[l] = [w]
        lemmas[l] = 1
        
# load all the words from big model
words = model_big.wv.index2word
words_lower = [w.lower() for w in words]

964564
1621239


/home/dzon/kajo/kajo_semexp_core/lib/python3.6/site-packages/ipykernel_launcher.py:29: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
/home/dzon/kajo/kajo_semexp_core/lib/python3.6/site-packages/ipykernel_launcher.py:30: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


In [7]:
# sort the lemmas from most frequent
sorted_lemmas = sorted(lemmas.items(), key=operator.itemgetter(1), reverse=True)
# trying first words
sorted_lemmas[:20]

[('dobry', 96),
 ('wielki', 94),
 ('zły', 90),
 ('mały', 89),
 ('wysoki', 83),
 ('używać', 83),
 ('być', 80),
 ('młody', 79),
 ('mieć', 78),
 ('ważny', 78),
 ('prowadzić', 76),
 ('zrobić', 75),
 ('widzieć', 74),
 ('chcieć', 73),
 ('on', 72),
 ('zmienić', 72),
 ('stać', 70),
 ('nowy', 70),
 ('pisać', 70),
 ('wykonywać', 70)]

In [8]:
# create vectors for the small models - average vectors for lemmas
# process only first MAX_WORDS in the big model
tmp_words = [None] * len(model_big.wv.index2word)
tmp_vecs = [None] * len(model_big.wv.index2word) 
for i,(word,lemma) in enumerate(all_words_lemmas[:MAX_WORDS]):
    lemma_words_list = big_lemma_word_dic[lemma]
    # add only lemmas made from at least 2 words
    if len(lemma_words_list)<2:
        continue
    tmp_vectors = []
    tmp_indexes = []
    error_with_word = False
    for word2 in lemma_words_list:
        try:
            tmp_vectors.append(model_big[word2])  
            tmp_indexes.append(model_big.vocab[word2].index)
        except:
            print(i, "error with ", word, " lemma ", lemma, "words", lemma_words_list)
            error_with_word = True
            continue
    if not error_with_word:
        try:
            tmp_words[min(tmp_indexes)] = lemma
            tmp_vecs[min(tmp_indexes)] = list(np.average(tmp_vectors, axis=0))
        except:
            print(i, "error with lemma ", lemma, " - words", lemma_words_list, " indexes: ", tmp_indexes)
            # note - we are processing first MAX_WORDS from the model after the filter out of the "non-words"
            # so some of the last words in that range may refer to words on bigger index than MAX_WORDS

# filter the lists (some positions remained None)
final_words = [i for i in tmp_words if i] 
final_vecs = [i for i in tmp_vecs if i != None] 

print(len(final_vecs), len(final_words))
len(final_vecs[0])  

del tmp_vecs
del tmp_words

/home/dzon/kajo/kajo_semexp_core/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until
/home/dzon/kajo/kajo_semexp_core/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  after removing the cwd from sys.path.


118739 118739


In [9]:
# create new gensim word2vec model with the vectors:
new_vector_space = gensim.models.keyedvectors.Word2VecKeyedVectors(300)
new_vector_space.add(final_words,final_vecs)

In [10]:
# test the model
pprint(new_vector_space.most_similar('ojciec'))
pprint(new_vector_space.most_similar_cosmul(positive=['kobieta', 'król'], negative=['mężczyzna']))

[('syn', 0.789566159248352),
 ('matka', 0.7510485053062439),
 ('brat', 0.7382024526596069),
 ('ojcowie', 0.6814154386520386),
 ('tato', 0.6785798072814941),
 ('mąż', 0.6568713784217834),
 ('dziadek', 0.6566153764724731),
 ('jezus', 0.6492627859115601),
 ('tatuś', 0.6465122103691101),
 ('bóg', 0.6369063854217529)]
[('królowa', 0.9656504988670349),
 ('królestwo', 0.8727908730506897),
 ('królewna', 0.8572472333908081),
 ('korona', 0.853415310382843),
 ('królowy', 0.8514885306358337),
 ('władca', 0.8511689901351929),
 ('księżniczka', 0.8503044843673706),
 ('królewski', 0.8411331176757812),
 ('tron', 0.8315914273262024),
 ('dama', 0.8280190825462341)]


In [11]:
# save the model
new_vector_space.save_word2vec_format(model_filename, binary=True)

In [ ]:
#model = KeyedVectors.load_word2vec_format(model_filename, binary=True)
model = new_vector_space
words_model = model.wv.index2word

In [12]:
# load topic listst
with open(nolemma_list_file) as fp:
    topics_nolemma = fp.readlines() 
print(len(topics_nolemma))
    
# load the lemma list
# with open(lemma_list_file) as fp:
#    topics = fp.readlines()
# print(len(topics))

topics = []
# generate the lemma list
for topic in tqdm(topics_nolemma):
    topic = re.sub('\W',' ',topic)
    topic = re.sub('\ +',' ',topic)
    doc = nlp(topic.strip().lower())
    lemmas = []
        
    for token in doc:
        if token.is_stop:
            continue
        lemmas.append(token.lemma_)    
    topics.append("_".join(lemmas))
    
print(len(topics))    
    
#write the lemma topic list
with open(lemma_list_file, 'wt', encoding='utf8') as f:
    for topic_lemma in topics:
        f.write(topic_lemma + "\n")    

  0%|          | 6/5308 [00:00<01:29, 59.22it/s]

5308


100%|██████████| 5308/5308 [00:50<00:00, 106.15it/s]


5308


In [13]:
#try if the topic-lemma words exist in the models  

words_to_manual_add = {}

for i, (topic, topic_nolemma) in enumerate(zip(topics, topics_nolemma)):
    # process the nolemma topic so we can see which word was lemmatized to which lemma
    topic_nolemma = re.sub('\W',' ',topic_nolemma)
    topic_nolemma = re.sub('\ +',' ',topic_nolemma)
    doc = nlp(topic_nolemma.strip().lower())
    nolemmas = []
    for token in doc:
        if token.is_stop:
            continue
        else:
            nolemmas.append(token.text)  
    
    topic = topic.strip()
    topic_words = topic.split('_')
    if len(topic_words) != len(nolemmas):
        print("!!!!! error - not equal length: ", nolemmas, topic_words)
        continue
    for word, word_nolemma in zip(topic_words, nolemmas):
        #print(word)
        found = 0
        if word in words_model:
            found = 1
            #print("l1")
        elif word in big_lemmas_set:
            found = 1
            print("l2", word)
        elif word in words_lower:
            found = 1
            print("l3", word)
        # nolemma words are in PL case probably not needed and all lemmas exist in lemma list
        elif word_nolemma in words_model:
            found = 1
            print("w1")
        elif word_nolemma in big_lemmas_set:
            found = 1
            print("w2")
        elif word_nolemma in words_lower:
            found = 1
            print("w3")

        if found == 0:
            print(i, 'needs to be added manually: ', word, " - ", word_nolemma, " - ", topic_nolemma.strip())
            if word not in words_to_manual_add:
                words_to_manual_add[word] = topic_nolemma.strip()
                    
print(len(words_to_manual_add))          

4 needs to be added manually:  wykształacenie  -  wykształacenie  -  wykształacenie ogólne
l2 wymywanie
142 needs to be added manually:  agrobiotechnologia  -  agrobiotechnologia  -  agrobiotechnologia
l2 hygiena
l2 drenujące
l2 renaturalizacja
l2 przesiąkanie
280 needs to be added manually:  przesiąkająca  -  przesiąkająca  -  woda przesiąkająca
295 needs to be added manually:  fianansami  -  fianansami  -  zarządzanie fianansami
436 needs to be added manually:  samoooczyszczanie  -  samoooczyszczanie  -  samoooczyszczanie się gleby
l2 drenowanie
l2 warstwicowy
l2 ugorować
l2 geosfera
l2 mikroklimatyczny
l2 uprawowo
l2 biogeograficzny
l2 sanitacja
l2 stabilnośc
l3 nawadnianych
l2 słonawa
l2 odsalanie
l3 antarktyki
l2 ekologizacja
851 needs to be added manually:  wpólnoty  -  wpólnoty  -  finanse Wpólnoty Europejskiej
l2 pokopalnianych
931 needs to be added manually:  rachunkowść  -  rachunkowść  -  rachunkowść księgowa
937 needs to be added manually:  samorządzeniu  -  samorządzeniu  

4004 needs to be added manually:  sanitarnie  -  sanitarnie  -  składowisko zabezpieczone sanitarnie
l2 chagasa
4025 needs to be added manually:  tarasowaty  -  tarasowaty  -  ogród tarasowaty
4026 needs to be added manually:  halogenowane  -  halogenowane  -  węglowodory halogenowane
l2 galwanotechnik
l2 atrazyna
4038 needs to be added manually:  dielsowskich  -  dielsowskich  -  paliwo do silników dielsowskich
l2 renaturyzacja
4058 needs to be added manually:  antyhałasowy  -  antyhałasowa  -  bariera antyhałasowa
4087 needs to be added manually:  parlmentarny  -  parlmentarny  -  raport parlmentarny
l2 synekologia
l2 bitumin
l2 upłynnianie
l2 wysuszanie
l2 lipofilowa
l2 imisja
l2 przekraczaniu
l2 dieslowski
l2 wymywanie
l2 toczeniem
l2 paleoekologia
l2 spiekanie
l2 powstajace
4230 needs to be added manually:  członowiec  -  członowce  -  członowce
l2 ultrafiltracja
l2 półmetal
l2 nitrozwiązek
4276 needs to be added manually:  mikroklimatologia  -  mikroklimatologia  -  mikroklimatol

5274 needs to be added manually:  geotechnologia  -  geotechnologia  -  geotechnologia
5276 needs to be added manually:  haloform  -  haloform  -  haloform
5278 needs to be added manually:  poddźwięk  -  poddźwięk  -  poddźwięk
5280 needs to be added manually:  lizymetryczny  -  lizymetryczny  -  pomiar lizymetryczny
5282 needs to be added manually:  mikroekosystem  -  mikroekosystem  -  mikroekosystem
l2 mikrofiltracja
l2 przenawożenie
l2 acetyl
5289 needs to be added manually:  fitomasa  -  fitomasa  -  fitomasa
l2 stopochodny
5292 needs to be added manually:  dwubenzofurany  -  dwubenzofurany  -  dwubenzofurany polichlorowane
5293 needs to be added manually:  autoekologia  -  autoekologia  -  autoekologia
l2 słoniowaty
l2 bioreaktor
141


In [14]:
# for manual correcting of words
pprint(words_to_manual_add)

{'administacyjnych': 'uzasadnienie decyzji administacyjnych',
 'administarcyjnych': 'dostęp do dokumentów administarcyjnych',
 'adsorbcja': 'adsorbcja',
 'agrobiotechnologia': 'agrobiotechnologia',
 'amonifikacja': 'amonifikacja',
 'antyhałasowy': 'bariera antyhałasowa',
 'atmosfry': 'emisja do atmosfry',
 'autoekologia': 'autoekologia',
 'azbestocement': 'azbestocement',
 'azotoorganiczne': 'związki azotoorganiczne',
 'bakteriocyd': 'bakteriocyd',
 'bepieczeństwa': 'analiza bepieczeństwa',
 'bezhałasowa': 'technologia bezhałasowa',
 'beztelnowym': 'oczyszczanie w procesie beztelnowym',
 'biochemicza': 'reakcja biochemicza',
 'chloroetylen': 'chloroetylen',
 'chlorofenol': 'chlorofenol',
 'chlorofluorowęglowodór': 'zakaz używania chlorofluorowęglowodorów i halonów',
 'chorologia': 'chorologia',
 'cyjanian': 'cyjanian',
 'cynoorganiczne': 'związki cynoorganiczne',
 'członowiec': 'członowce',
 'degazyfikacja': 'degazyfikacja odpadów',
 'dielsowskich': 'paliwo do silników dielsowskich',
 

In [15]:
# the lemmas not in model were manually checked if they exist in other models in some variation
word = "antarktyki"
print(word)
try:
    model[word]
    print('model')
    pprint(model.most_similar(word))
except:
    pass

if word in big_lemmas_set:
    print('model_lemma_big', big_lemma_word_dic[word])

try:
    model_big[word]
    print('model big')
    pprint(model_big.most_similar(word))
except:
    pass

# if the original word with uppercase
if word in words_lower_orig:
    print('model big LOWER', words_lower_orig[word])
    pprint(model_big.most_similar(words_lower_orig[word]))
    

antarktyki
model big LOWER ['Antarktyki']
[('Antarktydy', 0.7341861724853516),
 ('Arktyki', 0.6926125288009644),
 ('Antarktyk', 0.669875979423523),
 ('Antarktyce', 0.6694451570510864),
 ('Antarktykę', 0.6640111207962036),
 ('Antarktyd', 0.648389995098114),
 ('Antarktyka', 0.6459950804710388),
 ('arktyki', 0.6360539197921753),
 ('subantarktyki', 0.6342030167579651),
 ('Antarktyczne', 0.6149378418922424)]


In [16]:
# function which finds the word in all the available models and returns it's vector (average vector)
def get_vector(word):
    # word is in small model
    try:
        return model[word]
    except:
        pass
    
    # word is in lemmas from big model - return average if the lemma was result of more words
    if word in big_lemmas_set:
        lemma_words_list = big_lemma_word_dic[word]
        tmp_vectors = []
        for word2 in lemma_words_list:
            tmp_vectors.append(model_big[word2])  
        return np.average(tmp_vectors, axis=0)

    try:
        return model_big[word]
    except:
        pass
    
    # if the original word with uppercase
    if word in words_lower_orig:
        orig_words_list = words_lower_orig[word]
        tmp_vectors = []
        for word2 in orig_words_list:
            tmp_vectors.append(model_big[word2])  
        return np.average(tmp_vectors, axis=0)

        #return model_big[words_lower_orig[word]]
    
    return []

In [17]:
# check if the word vectors are the same
get_vector('Antarktyki') == get_vector('antarktyki')

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [18]:
# 2nd run - checking if there are still unavailable words in topics
excluded_topics = []
#try if the topic-lemma words exist in the model   

for i, (topic, topic_nolemma) in enumerate(zip(topics, topics_nolemma)):
    topic = topic.strip()
    topic_words = topic.split('_')
    found = 1
    for word in topic_words:
        #print(word)
       
        vector = get_vector(word)
        if vector == []:
            found = 0
            print('nonexistend word: ', word, " - ", topic_nolemma)

    if found == 0:
        print('!!! topic needs to be excluded because of word: ', word, " - ", word_nolemma, " - ", topic_nolemma.strip())
        if topic_nolemma.strip() not in excluded_topics:
            excluded_topics.append(topic_nolemma.strip())
                    
print(len(excluded_topics)) 

nonexistend word:  wykształacenie  -  wykształacenie ogólne

!!! topic needs to be excluded because of word:  ogólny  -  księgowość  -  wykształacenie ogólne
nonexistend word:  agrobiotechnologia  -  agrobiotechnologia

!!! topic needs to be excluded because of word:  agrobiotechnologia  -  księgowość  -  agrobiotechnologia
nonexistend word:  przesiąkająca  -  woda przesiąkająca

!!! topic needs to be excluded because of word:  przesiąkająca  -  księgowość  -  woda przesiąkająca
nonexistend word:  fianansami  -  zarządzanie fianansami

!!! topic needs to be excluded because of word:  fianansami  -  księgowość  -  zarządzanie fianansami
nonexistend word:  samoooczyszczanie  -  samoooczyszczanie się gleby

!!! topic needs to be excluded because of word:  gleba  -  księgowość  -  samoooczyszczanie się gleby
nonexistend word:  wpólnoty  -  finanse Wpólnoty Europejskiej

!!! topic needs to be excluded because of word:  europejski  -  księgowość  -  finanse Wpólnoty Europejskiej
nonexistend 

/home/dzon/kajo/kajo_semexp_core/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  del sys.path[0]


nonexistend word:  polno  -  krajobraz polno-leśny

!!! topic needs to be excluded because of word:  leśny  -  księgowość  -  krajobraz polno-leśny
nonexistend word:  odchlorowanie  -  odchlorowanie

!!! topic needs to be excluded because of word:  odchlorowanie  -  księgowość  -  odchlorowanie
nonexistend word:  spodowane  -  zakłócenie spodowane przez hałas

!!! topic needs to be excluded because of word:  hałas  -  księgowość  -  zakłócenie spodowane przez hałas
nonexistend word:  degazyfikacja  -  degazyfikacja odpadów

!!! topic needs to be excluded because of word:  odpady  -  księgowość  -  degazyfikacja odpadów
nonexistend word:  przyłowy  -  przyłowy

!!! topic needs to be excluded because of word:  przyłowy  -  księgowość  -  przyłowy
nonexistend word:  dokumnetacja  -  flora (dokumnetacja)

!!! topic needs to be excluded because of word:  dokumnetacja  -  księgowość  -  flora (dokumnetacja)
nonexistend word:  telededekcji  -  sprzęt używany w telededekcji

!!! topic needs to

nonexistend word:  chlorofluorowęglowodór  -  częściowo podstawione chlorowcem chlorofluorowęglowodory

!!! topic needs to be excluded because of word:  chlorofluorowęglowodór  -  księgowość  -  częściowo podstawione chlorowcem chlorofluorowęglowodory
nonexistend word:  chorologia  -  chorologia

!!! topic needs to be excluded because of word:  chorologia  -  księgowość  -  chorologia
nonexistend word:  teratogeneza  -  teratogeneza

!!! topic needs to be excluded because of word:  teratogeneza  -  księgowość  -  teratogeneza
nonexistend word:  chlorofenol  -  chlorofenol

!!! topic needs to be excluded because of word:  chlorofenol  -  księgowość  -  chlorofenol
nonexistend word:  ekopediatria  -  ekopediatria

!!! topic needs to be excluded because of word:  ekopediatria  -  księgowość  -  ekopediatria
nonexistend word:  paloeklimatologia  -  paloeklimatologia

!!! topic needs to be excluded because of word:  paloeklimatologia  -  księgowość  -  paloeklimatologia
nonexistend word:  r

In [33]:
# save topics containing words which are not present neither model - to be excluded from the topic list
print('excluded topics: ', len(excluded_topics))
pprint(excluded_topics)
with open(excluded_filename, 'wt', encoding='utf8') as f:
    for topic in excluded_topics:
        f.write(topic + "\n")

excluded topics:  148
['wykształacenie ogólne',
 'agrobiotechnologia',
 'woda przesiąkająca',
 'zarządzanie fianansami',
 'samoooczyszczanie się gleby',
 'finanse Wpólnoty Europejskiej',
 'rachunkowść księgowa',
 'udział w samorządzeniu',
 'przepływ niżówkowy',
 'krajobraz tarasowaty',
 'substancja ograniczna',
 'dolesienie',
 'ekoinformatyka',
 'analiza bepieczeństwa',
 'system melioracycjny',
 'rezewat morski',
 'usuwanie nierównomierności w rokładzie godzin pracy',
 'reakcja biochemicza',
 'inflitracja wody do ziemi',
 'międzynarodwe przepisy gospodarcze',
 'ekosystem lasów tropiklanych',
 'raport oceny oddzia lywania na środowisko',
 'lokalna elektorownia',
 'piecyk małoskalowy',
 'induktor małoskalowy',
 'dostęp do dokumentów administarcyjnych',
 'technologia bezhałasowa',
 'adsorbcja',
 'ośrodek sportu i rekreakcji',
 'cel makrogospodarczy',
 'skorupowce',
 'resuspensja osadu',
 'monitoring sejsmograficzny',
 'strefa międzypływowa',
 'rolnictwo z odłogowaniem',
 'minimizacja oddz

In [24]:
# create new lists which don't containt topics with non-existent words

# load list with excluded topics
with open(excluded_filename, 'rt', encoding='utf8') as fp:
    tmp_topics = fp.readlines()
excluded_set = set()
for topic in tmp_topics:
    excluded_set.add(topic.strip())

with open(nolemma_list_file, 'rt', encoding='utf8') as fp:
        topics_file = fp.readlines()

topics_final = []
topics_lemma = []

excluded = 0
for topic in topics_file:
    topic_orig = topic.strip()
    if topic_orig in excluded_set:
        print("EXCLUDE ", topic_orig)
        excluded += 1
        continue
    topic = re.sub('\W',' ',topic)
    topic = re.sub('\ +',' ',topic)
    doc = nlp(topic.strip().lower())
    lemmas = []

    for token in doc:
        if token.is_stop:
            continue
        lemmas.append(token.lemma_)    
    topics_lemma.append("_".join(lemmas))
    topics_final.append(topic_orig)

    print(topic_orig, ' --- ', "_".join(lemmas))

print('Excluded: ', excluded)
print('Final topics: ', len(topics_lemma), len(topics_final))
    
with open(lemma_list_file_excluded, 'wt', encoding='utf8') as f:
    for topic_lemma in topics_lemma:
        f.write(topic_lemma + "\n")
with open(nolemma_list_file_excluded, 'wt', encoding='utf8') as f:
    for topic_final in topics_final:
        f.write(topic_final + "\n")  

lokalny rozwój  ---  lokalny_rozwój
rozwój obszarów wiejskich  ---  rozwój_obszar_wiejski
zarządzanie zasobami wodnymi  ---  zarządzanie_zasób_wodny
poziom wykształcenia  ---  poziom_wykształcenie
EXCLUDE  wykształacenie ogólne
brzeg  ---  brzeg
prawo wspólnotowe  ---  prawo_wspólnotowy
dziedzictwo kulturowe  ---  dziedzictwo_kulturowy
grunty rolne  ---  grunt_rolny
krajobraz rolniczy  ---  krajobraz_rolniczy
produkcja rolna  ---  produkcja_rolny
rozwój gospodarczy  ---  rozwój_gospodarczy
łańcuch pokarmowy  ---  łańcuch_pokarmowy
przemysł spożywczy  ---  przemysł_spożywczy
jakość żywności  ---  jakość_żywność
ocieplenie globalne  ---  ocieplenie_globalny
efekt szklarniowy  ---  efekt_szklarniowy
rozwój gospodarczy  ---  rozwój_gospodarczy
rynek pracy  ---  rynka_praca
dostęp do terenów  ---  dostęp_teren
scalenie gruntów  ---  scalenie_grunt
planowanie przestrzenne  ---  planowanie_przestrzenny
architektura krajobrazu  ---  architektura_krajobraz
planowanie krajobrazu  ---  planowanie

niekontrolowany rozwój miast  ---  niekontrolowany_rozwój_miasto
zasoby  ---  zasób
wody drenujące  ---  woda_drenujące
struktura regionalna  ---  struktura_regionalny
sprzęt rolniczy  ---  sprzęt_rolniczy
grupa społecznie i ekonomicznie upośledzona  ---  grupa_społecznie_ekonomicznie_upośledzony
infrastruktura ruchu drogowego  ---  infrastruktura_rucho_drogowy
system bezpieczeństwa  ---  systema_bezpieczeństwo
infrastruktura  ---  infrastruktura
odkażanie gleb  ---  odkażanie_gleba
pastwisko  ---  pastwisko
wzrost gospodarczy  ---  wzrost_gospodarczy
degradacja wód słodkich  ---  degradacja_wóda_słodki
żywność  ---  żywność
zakwaszanie gleb  ---  zakwaszanie_gleba
dziedzictwo biologiczne  ---  dziedzictwo_biologiczny
przepis Wspólnot Europejskich  ---  przepis_wspólnota_europejski
budynki gospodarcze  ---  budynek_gospodarczy
budynki przemysłowe  ---  budynek_przemysłowy
klimat  ---  klimat
przemysł kosmetyczny  ---  przemysł_kosmetyczny
hodowla zwierząt  ---  hodowla_zwierzę
użytkowa

woda wykorzystywana do konsumpcji  ---  woda_wykorzystywać_konsumpcja
budynek  ---  budynek
analiza wód  ---  analiza_wóda
analiza glebowa  ---  analiza_glebowy
efekt oddziaływania zanieczyszczeń  ---  efekt_oddziaływanie_zanieczyszczenie
pogarszanie stanu środowiska  ---  pogarszanie_stan_środowisko
proces fizyczny  ---  proces_fizyczny
ochrona zwierząt  ---  ochrona_zwierzę
ochrona zwierząt  ---  ochrona_zwierzę
ochrona lasu  ---  ochrona_las
życie zwierząt  ---  żyto_zwierzę
gwarancja jakości  ---  gwarancja_jakość
nawóz zielony  ---  nawóz_zielony
siedlisko miejskie  ---  siedlisko_miejski
system edukacyjny  ---  systema_edukacyjny
usługi komunalne  ---  usługa_komunalny
tradycyjna architektura  ---  tradycyjny_architektura
struktura populacji  ---  struktura_populacja
szkody spowodowane przez wodę  ---  szkoda_spowodować_woda
zanieczyszczenia miejskie  ---  zanieczyszczenie_miejski
proces technologiczny  ---  proces_technologiczny
wielkość przedsiębiorstwa  ---  wielkość_przedsięb

tendencje ludnościowe  ---  tendencja_ludnościowy
potrzeby mieszkaniowe  ---  potrzeba_mieszkaniowy
polityka produkcyjna  ---  polityk_produkcyjny
region biogeograficzny  ---  region_biogeograficzny
system komunikowania się  ---  systema_komunikowania
tereny budowlane  ---  teren_budowlany
rynek finansowy  ---  rynka_finansowy
proces społeczny  ---  proces_społeczny
ocena działań środowiskowych  ---  ocena_działanie_środowiskowy
technologia energetyczna  ---  technologia_energetyczny
typ gleby  ---  typ_gleba
różnorodność biologiczna  ---  różnorodność_biologiczny
odszkodowanie  ---  odszkodowanie
sanitacja miast  ---  sanitacja_miasto
przemysł chłodniczy  ---  przemysł_chłodniczy
ponowne wykorzystanie wody  ---  ponowny_wykorzystanie_woda
siły zbrojne  ---  siła_zbrojny
szkodniki rolne  ---  szkodnik_rolny
obszar przyrody niezakłóconej przez człowieka  ---  obszar_przyroda_zakłócić_człowiek
działalność gospodarcza  ---  działalność_gospodarczy
warunki społeczne  ---  warunek_społeczny

przekształcanie krajobrazu  ---  przekształcanie_krajobraz
woda gruntowa  ---  woda_gruntowy
nauki przyrodnicze  ---  nauka_przyrodniczy
ustawa  ---  ustawa
ekosystem miejski  ---  ekosystem_miejski
gospodarowanie zapasami  ---  gospodarowanie_zapasy
polityka dotycząca udzielania pomocy socjalnej i humanitarnej  ---  polityk_dotyczyć_udzielania_pomoc_socjalny_humanitarny
pomoc na rzecz rozwoju  ---  pomoc_rzecz_rozwój
fundusz  ---  fundusz
sterowanie ryzykiem  ---  sterowanie_ryzyko
tlenek węgla  ---  tlenek_węgiel
interwencja na lądzie stałym  ---  interwencja_ląd_stały
zwierzęta rzeźne  ---  zwierzę_rzeźny
ubój zwierząt  ---  ubój_zwierzę
rozmieszczenie ludności  ---  rozmieszczenie_ludność
choroby zwierząt  ---  choroba_zwierzę
użytkowanie terenu  ---  użytkowanie_teren
wody (geografia)  ---  woda_geografia
stowarzyszenie zawodowe  ---  stowarzyszenie_zawodowy
mobilność przestrzenna  ---  mobilność_przestrzenny
wędrówka zwierząt  ---  wędrówka_zwierzę
walka z erozją  ---  walka_eroz

...

porzucone tereny zanieczyszczone  ---  porzucić_teren_zanieczyścić
wrak  ---  wrak
wydawanie zezwoleń  ---  wydawanie_zezwolenie
obowiązek posiadania zezwoleń  ---  obowiązek_posiadanie_zezwolenie
analiza ryzyka  ---  analiza_ryzyko
nakładanie obrazów  ---  nakładanie_obraz
wełna  ---  wełn
ultrafiltracja  ---  ultrafiltracja
litosfera  ---  litosfera
statut  ---  statut
elektrownia cieplna  ---  elektrownia_cieplny
opłata za odpady  ---  opłata_odpad
ustawodawstwo dotyczące gospodarki odpadami  ---  ustawodawstwo_dotyczyć_gospodarka_odpady
półmetal  ---  półmetal
wydobycie  ---  wydobycie
działania zmierzające do zachowania pokoju  ---  działanie_zmierzać_zachowanie_pokój
osad wtórny  ---  osada_wtórny
ryzyko jądrowe  ---  ryzyka_jądrowy
fermentacja  ---  fermentacja
bateria  ---  bateria
gazy spalinowe  ---  gaza_spalinowy
nitrozwiązki  ---  nitrozwiązek
pociąg  ---  pociąg
trawniki  ---  trawnik
bezprawie  ---  bezprawie
szkło  ---  szkło
aerozol azbestu  ---  aerozol_azbest
wycinan

sklep  ---  sklep
EXCLUDE  fenole halogenowane
kanion  ---  kanion
opodatkowanie  ---  opodatkowanie
odszkodowanie  ---  odszkodowanie
obowiązek etykietowania  ---  obowiązek_etykietowania
przelew  ---  przelew
pora roku  ---  por
klimatyzacja  ---  klimatyzacja
żywica  ---  żywica
spaliny silników lotniczych  ---  spaliny_silnik_lotniczy
tłuszcz jadalny  ---  tłuszcza_jadalny
wskaźnik promieniotwórczy  ---  wskaźnik_promieniotwórczy
laboratorium  ---  laboratorium
samochód-śmieciarka  ---  samochód_śmieciarka
wprowadzenie w życie postanowień Konferencji w Rio de Janeiro  ---  wprowadzenie_żyto_postanowienie_konferencja_rio_de_janeiro
poszukiwania geologiczne  ---  poszukiwanie_geologiczny
instytucjonalizacja  ---  instytucjonalizacja
wysypisko  ---  wysypisko
immunologia  ---  immunologia
bakteria  ---  bakteria
gotowość do ponoszenia wydatków  ---  gotowość_ponoszenia_wydatek
narażenie  ---  narażenie
odstraszanie  ---  odstraszać
hydrometeorologia  ---  hydrometeorologia
odpowiedzia

farba  ---  farba
polichlorek winylu  ---  polichlorek_winyl
wykrywacze  ---  wykrywacz
ryzyko  ---  ryzyka
ropucha  ---  ropucha
oczyszczanie  ---  oczyszczanie
eko-bilans  ---  eko_bilans
publiczna rozprawa  ---  publiczny_rozprawa
pojemnik na odpady  ---  pojemnik_odpad
tkanina  ---  tkanina
zezwolenie po uprzedzeniu  ---  zezwolenie_uprzedzenie
EXCLUDE  zakaz używania chlorofluorowęglowodorów i halonów
toksykologia  ---  toksykologia
reaktor  ---  reaktor
paliwo z odpadów komunalnych  ---  paliwo_odpady_komunalny
konsultacja  ---  konsultacja
śluza  ---  śluza
broń  ---  broń
polityka  ---  polityk
organizm nie będący obiektem zwalczania  ---  organizm_być_obiekt_zwalczanie
trzy główne zasady ochrony środowiska  ---  trzy_główny_zasada_ochrona_środowisko
kwiat  ---  kwiat
geofizyka  ---  geofizyk
substancje teratogenne  ---  substancja_teratogenny
absorpcja (ekspozycja)  ---  absorpcja_ekspozycja
podstawienie  ---  podstawienie
substancja niebezpieczna  ---  substancja_bezpieczny
k

termity  ---  termit
EXCLUDE  częściowo podstawione chlorowcem chlorofluorowęglowodory
mutant  ---  mutant
destylacja  ---  destylacja
wulkanologia  ---  wulkanologia
lampa oświetleniowe  ---  lampa_oświetleniowy
włókno szklane  ---  włókno_szklany
beryl  ---  beryl
trawienie  ---  trawienie
decybel  ---  decybel
piec  ---  piec
aminy  ---  amina
geotechnika  ---  geotechnik
feromon  ---  feromon
izotopy  ---  izotop
gatunek wymarły  ---  gatunek_wymarły
atrament  ---  atrament
glacjologia  ---  glacjologia
platyna  ---  platyna
przesiewanie  ---  przesiewanie
uraz  ---  uraza
test na kancerogenność  ---  test_kancerogenność
spalarnie odpadów  ---  spalarnia_odpady
powłoka  ---  powłoka
fenole  ---  fenol
ptak śpiewający  ---  ptak_śpiewać
pojemnik nienadający się do ponownego wykorzystania  ---  pojemnik_nadawać_ponowny_wykorzystanie
EXCLUDE  chorologia
marmur  ---  marmur
EXCLUDE  teratogeneza
represja  ---  represja
płazy ogoniaste  ---  płaza_ogoniasty
radon  ---  radon
uszczelnian

zarządzanie ryzykiem powodziowym  ---  zarządzanie_ryzyko_powodziowy
odpady spożywcze  ---  odpad_spożywczy
bioróżnorodność lasów  ---  bioróżnorodność_las
tereny leśne  ---  teren_leśny
zużycie paliw kopalnych  ---  zużycie_paliwo_kopalny
żywność funkcjonalna  ---  żywność_funkcjonalny
zielona gospodarka  ---  zielony_gospodarka
emisja gazów cieplarnianych  ---  emisja_gazy_cieplarniany
biomasa naziemna  ---  biomasa_naziemny
dyrektywa siedliskowa  ---  dyrektywa_siedliskowy
stres termiczny  ---  stres_termiczny
ciężki olej opałowy  ---  ciężki_olej_opałowy
czapa lodowa  ---  czapa_lodowy
ubytek lodu  ---  ubytek_lód
lądolód  ---  lądolód
bilans masy lądolodu  ---  bilans_masa_lądolód
degradacja gleby  ---  degradacja_gleba
dyrektywa w sprawie składowania odpadów  ---  dyrektywa_sprawa_składowania_odpady
paliwo okrętowe  ---  paliwo_okrętowy
dyrektywa ramowa w sprawie strategii morskiej  ---  dyrektywa_ramowy_sprawa_strategia_morski
wytwarzanie odpadów komunalnych  ---  wytwarzanie_od

In [25]:
# load the lemma list
with open(lemma_list_file_excluded) as fp:
    topics = fp.readlines()

#try if the topic-lemma words exist in the model     
new_vectors = []
new_words = []
for i,topic in enumerate(topics):
    #print(i, topic)
    topic = topic.strip()
    topic_words = topic.split('_')
    #if len(topic_words)<2:
    #    continue
    tmp_vectors = []
    for word in topic_words:
        
        try:
            tmp_vectors.append(model[word])
        except: 
            print('\nNOT in vocabulary: ', word)
            vector = get_vector(word)
            if vector != []:
                tmp_vectors.append(vector)
                if word not in new_words:
                    print('append ', word)
                    new_vectors.append(vector)
                    new_words.append(word)   
            else:
                raise Warning(f"XXX {word} {topic}")
                print("XXX", word, topic) 
    #if word not in new_words:
    #    new_vectors.append(list(np.average(tmp_vectors, axis=0)))
    #    new_words.append(word)        
    if len(tmp_vectors) > 1:
        if topic not in new_words:
   
            print('append ', topic)
            new_vectors.append(list(np.average(tmp_vectors, axis=0)))
            new_words.append(topic)
        else:
            print('doubled TOPIC: ', topic)    
    else:
        print('--- only one word, not a new topic>', topic)

append  lokalny_rozwój
append  rozwój_obszar_wiejski
append  zarządzanie_zasób_wodny
append  poziom_wykształcenie
--- only one word, not a new topic> brzeg
append  prawo_wspólnotowy
append  dziedzictwo_kulturowy
append  grunt_rolny
append  krajobraz_rolniczy
append  produkcja_rolny
append  rozwój_gospodarczy
append  łańcuch_pokarmowy
append  przemysł_spożywczy
append  jakość_żywność
append  ocieplenie_globalny
append  efekt_szklarniowy
doubled TOPIC:  rozwój_gospodarczy
append  rynka_praca
append  dostęp_teren
append  scalenie_grunt
append  planowanie_przestrzenny
append  architektura_krajobraz
append  planowanie_krajobraz
doubled TOPIC:  planowanie_przestrzenny
doubled TOPIC:  planowanie_przestrzenny
append  środowisko_morski
append  ochrona_przyroda
doubled TOPIC:  ochrona_przyroda
append  rolnictwo_ekologiczny
doubled TOPIC:  planowanie_przestrzenny
append  nauczanie_podstawowy
append  przemysł_samochodowy
append  jakość_życie
append  rozwój_regionalny
append  gospodarka_zasób
appen

/home/dzon/kajo/kajo_semexp_core/lib/python3.6/site-packages/ipykernel_launcher.py:22: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


append  żywność_przetworzyć
append  regulacja_prawny_dotyczyć_przetwórstwo_wykorzystanie_pasza
append  pomnik_przyroda
append  gospodarka_narodowy
append  izba_niski_parlament
append  opinia_publiczny

NOT in vocabulary:  strefowanie
append  strefowanie
append  strefowanie_przemysł
append  strefa_gospodarczy
append  emisja_proces_przemysłowy
append  sprzęt_rybacki
append  krajobraz_naturalny
append  obywatelstwo_środowiskowy

NOT in vocabulary:  woluntariuszy
append  woluntariuszy
append  praca_woluntariuszy
append  proces_atmosferyczny
append  dostęp_dobro_kultura
append  prawo_dyscyplina_naukowy
append  lokalny_rucho_komunikacyjny
append  lasa_mieszany
append  ochrona_wóda_gruntowy
append  przetwarzanie_obraz
append  pasza_zwierzęcy
append  technikum_informacyjny
append  kontrola_urodziny
append  budynek_ministerstwo
append  przemysł_budowlany
append  natężenie_hałas
append  przestępstwo_przeciwko_środowisko
append  warunek_praca
append  ochrona_gatunek
append  obszar_chronić
append 

append  pracownik_administracja
append  produkt_petrochemiczny
append  zanieczyszczenie_rzeka
append  monitoring_poziom_hałas
--- only one word, not a new topic> napowietrzanie
--- only one word, not a new topic> wywłaszczenie
append  struktura_wykorzystanie_energia

NOT in vocabulary:  biogeochemia
append  biogeochemia
--- only one word, not a new topic> biogeochemia
--- only one word, not a new topic> szron
append  ekologia_roślina
append  medycyna_praca
--- only one word, not a new topic> rurociąg
append  uzgodnienie_międzynarodowy
--- only one word, not a new topic> palestra
append  przedsięwzięcie_pilotowy
append  pożar_trawa
append  zagrożenie_hałas
append  zapas_energia
append  zanieczyszczenie_ujście_rzeka
--- only one word, not a new topic> ściółka
append  skład_atmosfera
append  ukształtowanie_teren
append  inżynieria_genetyczny
--- only one word, not a new topic> higiena
append  infrastruktura_turystyczny
append  dyskusja_naukowy
--- only one word, not a new topic> monitorin

append  ładunek_metal_ciężki
append  odpad_szklany
append  składowanie_odpady_teren
append  aktywny_udział
--- only one word, not a new topic> hałas
append  zbiornik_retencyjny
append  finansowy_aspekt_ograniczenie_emisja
append  teren_zewnętrzny
append  wymiana_informacja
append  informacja_prewencyjny

NOT in vocabulary:  kompostowania
append  kompostowania
append  odpad_nadawać_kompostowania
--- only one word, not a new topic> odpad
append  kompost_osady_ściekowy
--- only one word, not a new topic> czujnik
--- only one word, not a new topic> waleń
append  wzorcowanie_sprzęt_pomiarowy
append  czynnik_fotochemiczny
append  sprzęt_urządzenie_redukcja_zanieczyszczenie
append  okres_adaptacja
append  biologiczny_oczyszczanie_gaz
append  zło_metalowy
append  zależność_ziemia_słońce
--- only one word, not a new topic> taksonomia
--- only one word, not a new topic> schronisko
append  skażenie_ołów
append  papier_nadawać_ponowny_przetworzenie

NOT in vocabulary:  obrastanie
append  obrastani

--- only one word, not a new topic> marmur
--- only one word, not a new topic> represja
append  płaza_ogoniasty
--- only one word, not a new topic> radon
--- only one word, not a new topic> uszczelnianie

NOT in vocabulary:  odsiarczanie
--- only one word, not a new topic> odsiarczanie
--- only one word, not a new topic> workowaty
--- only one word, not a new topic> pchła
--- only one word, not a new topic> zasadowość
append  trasa_narciarski

NOT in vocabulary:  spazmatyczny
append  spazmatyczny
append  spazmatyczny_kaszel
--- only one word, not a new topic> silos
--- only one word, not a new topic> lepkość
--- only one word, not a new topic> mykologia
append  prawo_pierwokup_ziemia
--- only one word, not a new topic> steroid
--- only one word, not a new topic> fotografia

NOT in vocabulary:  miksja
append  miksja
--- only one word, not a new topic> miksja
append  rezerwat_antropologiczny
--- only one word, not a new topic> plac
--- only one word, not a new topic> jon
--- only one wor

In [26]:
# add topics to model
model.add(new_words,new_vectors)

In [29]:
# save the model with new topics
model.save_word2vec_format(excluded_model_name, binary=True)
#reload the model  otherwise cannot find similarities fo newly added words
model = KeyedVectors.load_word2vec_format(excluded_model_name, binary=True)

In [30]:
# try the model
model.similar_by_vector('bezpieczeństwo_biologiczny')

[('biologiczny', 0.8508073687553406),
 ('bezpieczeństwo_ekologiczny', 0.8279608488082886),
 ('przetwarzanie_biologiczny', 0.8159598708152771),
 ('różnorodność_biologiczny', 0.8083740472793579),
 ('zanieczyszczenie_pochodzenie_biologiczny', 0.797834038734436),
 ('bezpieczeństwo', 0.7966911792755127),
 ('rozwój_biologiczny', 0.7939287424087524),
 ('skażenie_biologiczny', 0.7918446063995361),
 ('dziedzictwo_biologiczny', 0.7901501655578613),
 ('zagrożenie_chemiczny', 0.7849078178405762)]

In [31]:
# find most similar to topics to terms in the new model

# load the new lemma list
with open(lemma_list_file_excluded) as fp:
    topics = fp.readlines()

for i, topic in enumerate(topics):
    topic = topic.strip()
    print('\n-----\n', topic)
    try:
        print(i, model.similar_by_vector(topic))
    except:
        print(topic, 'NOT in model')


-----
 lokalny_rozwój
0 [('rozwój_regionalny', 0.9252409934997559), ('rozwój_kulturalny', 0.8870919346809387), ('rozwój_społeczny', 0.8860856294631958), ('rozwój', 0.8821917176246643), ('rozwój_środowiskowy', 0.880439281463623), ('zrównoważony_rozwój', 0.8750596046447754), ('rozwój_technologiczny', 0.8687906265258789), ('rozwój_gospodarczy', 0.8686202168464661), ('rozwój_przestrzenny', 0.8684259653091431), ('rozwój_obszar_wiejski', 0.8541167974472046)]

-----
 rozwój_obszar_wiejski
1 [('plan_rozwój_teren_wiejski', 0.9038332104682922), ('obszar_aktywizacja_gospodarczy', 0.8680236339569092), ('rozwój_przestrzenny', 0.8571187853813171), ('rozwój_regionalny', 0.8551408648490906), ('lokalny_rozwój', 0.8541167974472046), ('rozwój_środowiskowy', 0.8499091863632202), ('rozwój_społeczny', 0.849903404712677), ('przepis_dotyczyć_obszar_wiejski', 0.8448479175567627), ('rozwój_gospodarczy', 0.8419251441955566), ('obszar_podmiejski', 0.8397102355957031)]

-----
 zarządzanie_zasób_wodny
2 [('adminis

33 [('rozwój', 0.9285483360290527), ('lokalny_rozwój', 0.9252409338951111), ('rozwój_gospodarczy', 0.9132816791534424), ('rozwój_technologiczny', 0.9115635752677917), ('rozwój_przestrzenny', 0.907659113407135), ('rozwój_środowiskowy', 0.907010555267334), ('zrównoważony_rozwój', 0.9034761190414429), ('rozwój_społeczny', 0.9028984904289246), ('rozwój_kulturalny', 0.9025741815567017), ('planowanie_rozwój', 0.8749101758003235)]

-----
 gospodarka_zasób
34 [('zasób', 0.9216301441192627), ('gospodarowanie_zasób_naturalny', 0.920883059501648), ('zasób_rybołówstwo', 0.9032121896743774), ('zasób_mikrobiologiczny', 0.9005611538887024), ('zasób_odnawialny', 0.8954453468322754), ('zasób_naturalny', 0.8905857801437378), ('eksploatacja_zasób', 0.8886125087738037), ('zasób_biologiczny', 0.8856830596923828), ('zasób_klimatyczny', 0.878105640411377), ('rozmieszczenie_zasób', 0.8663594722747803)]

-----
 ludność_mieszkać_wieś
35 [('wieś', 0.8749904632568359), ('ludność_miejski', 0.7814706563949585), ('l

51 [('plan_strukturalny_plan_zagospodarowania_przestrzenny', 0.9838242530822754), ('plan_rozmieszczenie', 0.9076880216598511), ('plan_gospodarczy', 0.9073134660720825), ('plan', 0.9014554023742676), ('plan_regionalny', 0.8931297063827515), ('plan_rozwój', 0.8807157278060913), ('plan_zaopatrzenie', 0.8765048980712891), ('plan_zarządzanie', 0.8680224418640137), ('plan_rozwój_miasto', 0.8596804141998291), ('plan_rozwój_teren_wiejski', 0.8588440418243408)]

-----
 siła_roboczy
52 [('siła', 0.9070567488670349), ('siła_zbrojny', 0.8893948793411255), ('siła_wiatry', 0.8170669078826904), ('roboczy', 0.6971300840377808), ('prawo_dotyczyć_rucho_powietrzny', 0.6528314352035522), ('przepis_rucho_powietrzny', 0.644771933555603), ('działanie_wojskowy', 0.6444857120513916), ('cywilny_rucho_powietrzny', 0.631877064704895), ('społeczny_systema_wartość', 0.630103349685669), ('ekspozycja_człowiek_działanie_zanieczyszczenie', 0.6297981142997742)]

-----
 śródlądowy_transport_wodny
53 [('transport_wodny', 

68 [('instrument_ekonomiczny', 0.8902738690376282), ('finansowy', 0.8662150502204895), ('instrument', 0.865352988243103), ('instrument_polityczny', 0.8521612882614136), ('instrument_zarządzanie_gospodarczy', 0.8501774072647095), ('rynka_finansowy', 0.7975190877914429), ('proekologiczne_instrument_fiskalny', 0.7968073487281799), ('instrument_polityka_środowiskowy', 0.7941113114356995), ('europejski_fundusz_walutowy', 0.7807090282440186), ('międzynarodowy_fundusz_walutowy', 0.7759321331977844)]

-----
 urbanistyka
69 [('urbanistyczny', 0.7910866737365723), ('urbanista', 0.7676448225975037), ('architektura', 0.6806097030639648), ('architektura_krajobraz', 0.6718671917915344), ('urbanistycznej', 0.6349906325340271), ('inżynieria_miejski', 0.6344562768936157), ('architektoniczny', 0.6317218542098999), ('gospodarka_przestrzenny', 0.6244012117385864), ('urbanistyczno', 0.6240748763084412), ('przestrzenny', 0.6102374196052551)]

-----
 gospodarowanie_woda_powierzchniowy
70 [('woda_powierzchnio

100 [('ludność', 0.8871273994445801), ('rozmieszczenie_ludność', 0.8537948131561279), ('migracja_ludność', 0.8106777667999268), ('ludność_koczownicza', 0.8020396828651428), ('ludność_mieszkać_wieś', 0.781470537185669), ('spisa_ludność', 0.771808385848999), ('miejski', 0.7625207901000977), ('ludność_czynny_zawodowo_rolnictwo', 0.7576775550842285), ('ludność_aktywny_zawodowo', 0.7563489079475403), ('zanieczyszczenie_miejski', 0.7496024370193481)]

-----
 ugór
101 [('łąka', 0.6772482395172119), ('łaki', 0.6708625555038452), ('orać', 0.6634838581085205), ('zaorać', 0.6537710428237915), ('pastwisko', 0.6476352214813232), ('orny', 0.6433597207069397), ('łąka_podmorski', 0.624055027961731), ('nieużytki', 0.6238270998001099), ('chwast', 0.622331976890564), ('kosić', 0.6212335824966431)]

-----
 kształcenie_poziom_wysoki
102 [('poziom_wykształcenie', 0.9056825637817383), ('edukacja_poziom_średni', 0.8826112151145935), ('poziom', 0.8620444536209106), ('poziom_efektywność', 0.861824631690979), ('

...

5031 [('biomasa_podziemny', 0.9232970476150513), ('biomasa_nadziemny', 0.8704463243484497), ('biomasa_naziemny', 0.8515759110450745), ('biomasa', 0.8209062814712524), ('energia_biomasa', 0.7948108911514282), ('surowiec_odnawialny', 0.7344906330108643), ('całkowity_węgiel_organiczny', 0.7304750680923462), ('zużycie_paliwo_kopalny', 0.728392481803894), ('łączny_spalanie_odpady', 0.7189329862594604), ('odzysk_surowiec_odpadowy', 0.7112897634506226)]

-----
 niebieski_wzrost
5032 [('wzrost', 0.8803728222846985), ('wzrost_demograficzny', 0.8344689011573792), ('wzrost_gospodarczy', 0.8219356536865234), ('wzrost_koszt', 0.745525598526001), ('wzrostowy', 0.7126245498657227), ('wskaźnik_zrównoważony_rozwój', 0.6806047558784485), ('przyrost', 0.6712157130241394), ('niebieski', 0.6685053110122681), ('maksymalny_poziom_przypływ', 0.6662297248840332), ('niski', 0.6625094413757324)]

-----
 ilość_węgiel_związanego
5033 [('całkowity_węgiel_organiczny', 0.8454092741012573), ('dwutlenek_węgiel', 0.8438

5062 [('przestrzeń', 0.8318930864334106), ('przyrodniczy', 0.8291682600975037), ('interwencja_krajobraz_przyrodniczy', 0.8049051761627197), ('dziedzictwo_przyrodniczy', 0.7880731821060181), ('wartość_przyrodniczy', 0.7858601212501526), ('zasób_przyroda_ożywiony', 0.7794251441955566), ('przestrzeń_międzyplanetarny', 0.7617565989494324), ('obszar_przyroda_zakłócić_człowiek', 0.7578607797622681), ('przestrzeń_życiowy', 0.7526869773864746), ('ochrona_obszar_przyrodniczy', 0.7490066289901733)]

-----
 unijny_systema_handel_uprawnienie_emisja
5063 [('systema_handel_emisja', 0.942154049873352), ('handel_emisja', 0.8925608992576599), ('emisja_proces_przemysłowy', 0.8426903486251831), ('finansowy_aspekt_ograniczenie_emisja', 0.8424317836761475), ('uprawnienie_emisja', 0.8304742574691772), ('prawodawstwo_zakres_emisja_zanieczyszczenie', 0.8283895254135132), ('rejestr_emisja', 0.8235656023025513), ('sytuacja_zakres_emisja', 0.794211745262146), ('oddziaływanie_handel_środowisko', 0.792554140090942

5095 [('odpad_organiczny_ogród', 0.8336610198020935), ('park_przyrodniczy', 0.8063532710075378), ('park', 0.7970296740531921), ('odpad_przemysłowy', 0.7942032814025879), ('park_publiczny', 0.7910270094871521), ('odpad_handlowy', 0.7893005609512329), ('park_regionalny', 0.7812473773956299), ('odpad_komunalny', 0.7800147533416748), ('odpad_szpitalny', 0.7768168449401855), ('park_cysterna', 0.776564359664917)]

-----
 biostymulator
5096 [('stymulator', 0.6369027495384216), ('herbicyd', 0.5476523041725159), ('insektycyd', 0.5384361743927002), ('fungicyd', 0.5347088575363159), ('resweratrol', 0.5314341187477112), ('aktywator', 0.5194366574287415), ('bioreaktor', 0.5146454572677612), ('nawóz_nieorganiczny', 0.508723258972168), ('nawóz_fosforowy', 0.5086370706558228), ('antyutleniacz', 0.5023841857910156)]

-----
 rama_polityka
5097 [('rama', 0.9055236577987671), ('realizacja_polityka', 0.6380624175071716), ('ramy', 0.6259021162986755), ('polityka', 0.6249261498451233), ('wytyczna_polityka', 

5112 [('wspólnota_europejski', 0.8550618290901184), ('systema_prawny_wspólnota_europejski', 0.850861132144928), ('europejski_sieć_informacja_obserwacja_środowisko', 0.8325846791267395), ('budżet_wspólnota_europejski', 0.8320354223251343), ('przepis_wspólnota_europejski', 0.8197293281555176), ('orzeczenie_wspólnota_europejski', 0.8163818120956421), ('infrastruktura_społeczny', 0.8105473518371582), ('przepis_komisja_europejski_dotyczyć_gospodarka_środowisko_audyt', 0.8046773672103882), ('infrastruktura_informacyjny', 0.8038648366928101), ('gospodarka_przestrzenny', 0.7946920990943909)]

-----
 odpad_trafiać_składowisko
5113 [('odpad_bezpieczny', 0.8759855031967163), ('odpad_wielkogabarytowy', 0.8744312524795532), ('odpad', 0.8662943840026855), ('odpad_elektroniczny', 0.8591720461845398), ('odpad_specjalny', 0.857954740524292), ('odpad_produkcyjny', 0.8554607033729553), ('pojemnik_odpad', 0.8553526401519775), ('odpad_promieniotwórczy', 0.852698802947998), ('odpad_aktywny', 0.8520026206970

5144 [('koniowate', 0.7342456579208374), ('świniowaty', 0.7209368944168091), ('słoniowy', 0.7028947472572327), ('gołębiowaty', 0.6678065061569214), ('ropuchowaty', 0.6591813564300537), ('jeleniowaty', 0.6526098251342773), ('wołowaty', 0.6521458625793457), ('żabowaty', 0.6502227187156677), ('niedźwiedziowaty', 0.6465935111045837), ('kangurowaty', 0.6429702043533325)]

-----
 agroturystyka
5145 [('agroturystyczny', 0.8741563558578491), ('turystyka', 0.6751730442047119), ('pensjonat', 0.6436909437179565), ('letniskowy', 0.637076199054718), ('ekoturystyki', 0.6366342306137085), ('turystyka_górski', 0.6297637224197388), ('nocleg', 0.6249487400054932), ('noclegowy', 0.6226354837417603), ('turystyczny', 0.6150403022766113), ('zarządzanie_turystyka', 0.6130497455596924)]

-----
 pokaz
5146 [('pokazówa', 0.7499275803565979), ('pokazowy', 0.6481441259384155), ('wystawa', 0.6436386108398438), ('prezentacja', 0.6313809156417847), ('przedstawienie', 0.6300503015518188), ('projekcja', 0.615638196468